In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)
quiet_library(ArchR)

In [ ]:
cd8_na <- readRDS(file = 'cd8_naive_processed.rds')

# Trim low number clusters

In [ ]:
table(cd8_na$`wsnn_res.0.3`)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 6)
DimPlot(cd8_na, reduction = 'wnn.3.umap', group.by = 'wsnn_res.0.3', shuffle = TRUE, label = TRUE)

In [ ]:
cd8_na <- SetIdent(cd8_na, value = 'wsnn_res.0.3')
cd8_na <- FindSubCluster(object = cd8_na, cluster = 4, graph.name = 'wsnn', resolution = 0.2)
Idents(cd8_na) <- "sub.cluster"

In [ ]:
table(cd8_na$`sub.cluster`)

In [ ]:
cd8_na <- SetIdent(cd8_na, value = 'sub.cluster')
trim_cd8_na <- subset(cd8_na, idents = c('4_2'), invert = TRUE)

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 15)
p1 <- DimPlot(trim_cd8_na, reduction = 'wnn.3.umap', group.by = 'sub.cluster', label = TRUE, label.size = 6)
p2 <- DimPlot(trim_cd8_na, reduction = 'wnn.3.umap', group.by = 'pediatric_senior', shuffle = TRUE)
p3 <- DimPlot(trim_cd8_na, reduction = 'wnn.3.umap', group.by = 'age_cmv', shuffle = TRUE)
p4 <- FeaturePlot(trim_cd8_na, reduction = 'wnn.3.umap', features = 'percent.mt')
p1 + p2 + p3 + p4

# DEG Analysis

In [ ]:
trim_cd8_na <- SetIdent(trim_cd8_na, value = 'sub.cluster')
rna_markers <- FindAllMarkers(trim_cd8_na, assay = 'SCT', max.cells.per.ident = 500)

In [ ]:
rna_markers %>%
    group_by(cluster) %>%
    top_n(n = 10, wt = avg_log2FC) -> top10

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 15)
trim_cd8_na_subset <- subset(trim_cd8_na, downsample = 500)
DoHeatmap(trim_cd8_na_subset, features = top10$gene, assay = 'SCT') + NoLegend()

In [ ]:
# write.csv(rna_markers, file = '/home/jupyter/Organized_Gating_Analysis/03_Subset_Deep_Dives/CD8_Naive/CD8na_rna_markers_cluster.csv')

# Cluster / Cell Types

In [ ]:
trim_cd8_na <- SetIdent(trim_cd8_na, value = 'sub.cluster')
trim_cd8_na <- RenameIdents(trim_cd8_na, '0' = 'CD8 Naive-1',
                          '1' = 'CD8 Naive-2', '2' = 'CD8 Naive-3', '3' = 'CD8 Naive-4', 
                          '4_0' = 'CD8 MNP-1', '4_1' = 'CD8 SCM',
                          '5' = 'CD8 Naive-5',
                          '6' = 'CD8 Naive-6',
                          '7' = 'CD8 MNP-2',
                          '8' = 'CD8 MAIT')
trim_cd8_na$wnn_celltype <- Idents(trim_cd8_na)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 10)
DimPlot(trim_cd8_na, reduction = 'wnn.3.umap', group.by = 'wnn_celltype', label = TRUE, label.size = 8, repel = TRUE)

In [ ]:
trim_cd8_na <- SetIdent(trim_cd8_na, value = 'sub.cluster')
trim_cd8_na <- RenameIdents(trim_cd8_na, '0' = 'CD8 Naive',
                          '1' = 'CD8 Naive', '2' = 'CD8 Naive', '3' = 'CD8 Naive', 
                          '4_0' = 'CD8 MNP-1', '4_1' = 'CD8 SCM',
                          '5' = 'CD8 Naive',
                          '6' = 'CD8 Naive',
                          '7' = 'CD8 MNP-2',
                          '8' = 'CD8 MAIT')
trim_cd8_na$pure_celltype <- Idents(trim_cd8_na)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 10)
DimPlot(trim_cd8_na, reduction = 'wnn.3.umap', group.by = 'pure_celltype', label = TRUE, label.size = 8, repel = TRUE)

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 15)
p1 <- DimPlot(trim_cd8_na, reduction = 'umap', group.by = 'pure_celltype', label = TRUE)
p2 <- DimPlot(trim_cd8_na, reduction = 'adtumap', group.by = 'pure_celltype', label = TRUE)
p3 <- DimPlot(trim_cd8_na, reduction = 'atac_umap', group.by = 'pure_celltype', label = TRUE)
p4 <- DimPlot(trim_cd8_na, reduction = 'wnn.3.umap', group.by = 'pure_celltype', label = TRUE)
p1 + p2 + p3 + p4

# Save 

In [ ]:
saveRDS(trim_cd8_na, file = 'trim_cd8_na.rds')

In [ ]:
sessionInfo()

In [ ]:
# trim_cd8_na <- readRDS(file = '/home/jupyter/Organized_Gating_Analysis/Seurat_Objects/trim_cd8_na.rds')

In [ ]:
# saveRDS(trim_cd8_na@meta.data, file = '/home/jupyter/Organized_Gating_Analysis/03_Subset_Deep_Dives/CD8_Naive/metadata.rds')